In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from scipy import signal

import warnings
warnings.filterwarnings("ignore")

In [36]:
df = pd.read_csv("../Data/Final/STI0220_cov20_pop0220.csv", index_col = 0)
df.head()

,year,fips,cases_chlamydia,cases_early_non-primary_non-secondary_syphilis,cases_gonorrhea,cases_primary_and_secondary_syphilis,rate_per_100000_chlamydia,rate_per_100000_early_non-primary_non-secondary_syphilis,rate_per_100000_gonorrhea,rate_per_100000_primary_and_secondary_syphilis,...,pct_ind_transportation_and_warehousing_and_utilities,pct_ind_information,pct_ind_finance_and_insurance_and_real_estate_and_rental_and_leasing,pct_ind_professional_scientific_and_management_and_administrative_and_waste_management_services,pct_ind_educational_services_and_health_care_and_social_assistance,pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services,pct_ind_other_services_except_public_administration,pct_ind_public_administration,health_insurance_rate,poverty_rate
0,2000,45001,61.0,1.0,57.0,2.0,232.6,3.8,217.4,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,22001,0.0,8.0,0.0,11.0,0.0,13.6,0.0,18.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,51001,153.0,2.0,88.0,0.0,398.7,5.2,229.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,16001,644.0,0.0,33.0,1.0,212.3,0.0,10.9,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,19001,8.0,0.0,0.0,0.0,97.6,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
cols = ['cases_gonorrhea','cases_chlamydia','cases_early_non-primary_non-secondary_syphilis',
        'cases_primary_and_secondary_syphilis', 'year']

df = df[df['year'] >= 2002]
fips = df['fips'].unique()
print(len(fips))
print(df.dtypes)

3214
year                                                                               int64
fips                                                                               int64
cases_chlamydia                                                                  float64
cases_early_non-primary_non-secondary_syphilis                                   float64
cases_gonorrhea                                                                  float64
                                                                                  ...   
pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services    float64
pct_ind_other_services_except_public_administration                              float64
pct_ind_public_administration                                                    float64
health_insurance_rate                                                            float64
poverty_rate                                                                     float64
Length: 180, dty

In [38]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_chlamydia"]]
    #print(df_model.shape)
    df_model = df_model[(df_model["year"] <= 2019)]
    #df_model['cases_chlamydia'] = signal.detrend(df_model.cases_chlamydia)
    actual_value = df_model.iloc[-1, df_model.columns.get_loc('cases_chlamydia')] 
    df_plot = df_model
    #print(df_model.head(20))
    #print(actual_value)
    
    model = ARIMA(df_model["cases_chlamydia"], order=(2,2,0))
    model_fit = model.fit()
    
    pred = model_fit.predict(18).to_frame()
    fitted_value = pred.iloc[0][0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)
        
    #print(model_fit.resid)
    #print(i , ':    ', fitted_value)
    #print(df_plot.head(20))
    #print(df_model.head(20))

45001 :     125.0  &  162.4380546344182
22001 :     345.0  &  326.6195716951096
51001 :     232.0  &  236.3179300099684
16001 :     2203.0  &  2325.0006252934613
19001 :     27.0  &  26.857422588747735
21001 :     50.0  &  51.91945979542947
29001 :     89.0  &  98.1976042300843
40001 :     104.0  &  105.40246053289914
8001 :     2903.0  &  3057.492227502474
19003 :     14.0  &  8.98234894501892
16003 :     3.0  &  3.337908094764761


In [39]:
dff = pd.DataFrame()
dff['fips'] = fips
dff['act_values_chlamydia'] = act_values
dff['pred_values_chlamydia'] = pred_values
dff['errors_chlamydia'] = errors

In [40]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_gonorrhea"]]
    #print(df_model.shape)
    df_model = df_model[(df_model["year"] <= 2019)]
    #df_model['cases_gonorrhea'] = signal.detrend(df_model.cases_gonorrhea)
    actual_value = df_model.iloc[-1, df_model.columns.get_loc('cases_gonorrhea')] 
    df_plot = df_model
    #print(df_model.head(20))
    #print(actual_value)
    
    model = ARIMA(df_model["cases_gonorrhea"], order=(2,2,0))
    model_fit = model.fit()
    
    pred = model_fit.predict(18).to_frame()
    fitted_value = pred.iloc[0][0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)

45001 :     58.0  &  68.09850356711293
22001 :     112.0  &  111.33681586666982
51001 :     44.0  &  65.49698558550948
16001 :     580.0  &  634.838528057362
19001 :     0.0  &  9.766283692692056
21001 :     15.0  &  17.771833079162903
29001 :     37.0  &  46.40430164084448
40001 :     39.0  &  17.82095435504486
8001 :     960.0  &  1130.8054490906948
19003 :     1.0  &  1.501097500743387
16003 :     0.0  &  0.22071271257658232


In [41]:
dff['act_values_gonorrhea'] = act_values
dff['pred_values_gonorrhea'] = pred_values
dff['errors_gonorrhea'] = errors

In [42]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_early_non-primary_non-secondary_syphilis"]]
    #print(df_model.shape)
    df_model = df_model[(df_model["year"] <= 2019)]
    #df_model['cases_early_non-primary_non-secondary_syphilis'] = signal.detrend(df_model['cases_early_non-primary_non-secondary_syphilis'])
    actual_value = df_model.iloc[-1, df_model.columns.get_loc('cases_early_non-primary_non-secondary_syphilis')] 
    df_plot = df_model
    #print(df_model.head(20))
    #print(actual_value)
    
    model = ARIMA(df_model["cases_early_non-primary_non-secondary_syphilis"], order=(2,2,0))
    model_fit = model.fit()
    
    pred = model_fit.predict(18).to_frame()
    fitted_value = pred.iloc[0][0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)
    #print(i , ':    ', fitted_value)
    #print(df_plot.head(20))
    #print(df_model.head(20))

45001 :     1.0  &  6.192247198206704
22001 :     3.0  &  5.845936778111619
51001 :     1.0  &  2.4300781056247276
16001 :     26.0  &  27.74185511035425
19001 :     0.0  &  0.0
21001 :     1.0  &  2.0000004515692313
29001 :     0.0  &  -1.0026371695744497e-12
40001 :     0.0  &  0.5237555556236135
8001 :     38.0  &  40.090152461596325
19003 :     0.0  &  1.5808460960505628
16003 :     0.0  &  0.0


In [43]:
dff['act_values_early_non-primary_non-secondary_syphilis'] = act_values
dff['pred_values_early_non-primary_non-secondary_syphilis'] = pred_values
dff['errors_early_non-primary_non-secondary_syphilis'] = errors

In [44]:
act_values = []
pred_values = []
errors = []

#fips = fips[:500]

for index, i in enumerate(fips):
    df1 = df[(df['fips']==i) & (df['year']>= 2002)].loc[:,cols]
    df_sum = df1.groupby('year').sum().reset_index()
    df_model = df_sum[["year", "cases_primary_and_secondary_syphilis"]]
    #print(df_model.shape)
    df_model = df_model[(df_model["year"] <= 2019)]
    #df_model['cases_primary_and_secondary_syphilis'] = signal.detrend(df_model['cases_primary_and_secondary_syphilis'])
    actual_value = df_model.iloc[-1, df_model.columns.get_loc('cases_primary_and_secondary_syphilis')] 
    df_plot = df_model
    #print(df_model.head(20))
    #print(actual_value)
    
    model = ARIMA(df_model["cases_primary_and_secondary_syphilis"], order=(2,2,0))
    model_fit = model.fit()
    
    pred = model_fit.predict(18).to_frame()
    fitted_value = pred.iloc[0][0]
    #print(df['year'].nunique())
    
    act_values.append(actual_value)
    pred_values.append(fitted_value)
    errors.append(actual_value - fitted_value)
    
    #print(model_fit.summary())
    if index <= 10:
        print(i , ':    ', actual_value, ' & ', fitted_value)
    #print(i , ':    ', fitted_value)
    #print(df_plot.head(20))
    #print(df_model.head(20))

45001 :     3.0  &  3.274323536518391
22001 :     7.0  &  7.172446161779563
51001 :     0.0  &  0.3363186032960791
16001 :     14.0  &  8.868490432554118
19001 :     0.0  &  0.43914528366550454
21001 :     0.0  &  -7.271994942066528e-30
29001 :     3.0  &  3.4076884291558684
40001 :     0.0  &  0.43914528366550454
8001 :     55.0  &  64.51687421159919
19003 :     0.0  &  0.0
16003 :     0.0  &  0.0


In [45]:
dff['act_values_primary_and_secondary_syphilis'] = act_values
dff['pred_values_primary_and_secondary_syphilis'] = pred_values
dff['errors_early_primary_and_secondary_syphilis'] = errors

In [46]:
dff.head(20)

,fips,act_values_chlamydia,pred_values_chlamydia,errors_chlamydia,act_values_gonorrhea,pred_values_gonorrhea,errors_gonorrhea,act_values_early_non-primary_non-secondary_syphilis,pred_values_early_non-primary_non-secondary_syphilis,errors_early_non-primary_non-secondary_syphilis,act_values_primary_and_secondary_syphilis,pred_values_primary_and_secondary_syphilis,errors_early_primary_and_secondary_syphilis
0,45001,125.0,162.438055,-37.438055,58.0,68.098504,-10.098504,1.0,6.192247e+00,-5.192247e+00,3.0,3.274324e+00,-2.743235e-01
1,22001,345.0,326.619572,18.380428,112.0,111.336816,0.663184,3.0,5.845937e+00,-2.845937e+00,7.0,7.172446e+00,-1.724462e-01
2,51001,232.0,236.317930,-4.317930,44.0,65.496986,-21.496986,1.0,2.430078e+00,-1.430078e+00,0.0,3.363186e-01,-3.363186e-01
3,16001,2203.0,2325.000625,-122.000625,580.0,634.838528,-54.838528,26.0,2.774186e+01,-1.741855e+00,14.0,8.868490e+00,5.131510e+00
4,19001,27.0,26.857423,0.142577,0.0,9.766284,-9.766284,0.0,0.000000e+00,0.000000e+00,0.0,4.391453e-01,-4.391453e-01
5,21001,50.0,51.919460,-1.919460,15.0,17.771833,-2.771833,1.0,2.000000e+00,-1.000000e+00,0.0,-7.271995e-30,7.271995e-30
6,29001,89.0,98.197604,-9.197604,37.0,46.404302,-9.404302,0.0,-1.002637e-12,1.002637e-12,3.0,3.407688e+00,-4.076884e-01
7,40001,104.0,105.402461,-1.402461,39.0,17.820954,21.179046,0.0,5.237556e-01,-5.237556e-01,0.0,4.391453e-01,-4.391453e-01
8,8001,2903.0,3057.492228,-154.492228,960.0,1130.805449,-170.805449,38.0,4.009015e+01,-2.090152e+00,55.0,6.451687e+01,-9.516874e+00
9,19003,14.0,8.982349,5.017651,1.0,1.501098,-0.501098,0.0,1.580846e+00,-1.580846e+00,0.0,0.000000e+00,0.000000e+00


In [47]:
print(dff.head(10))
dff.to_csv('../Data/Final/model_pred_values.csv')

    fips  act_values_chlamydia  pred_values_chlamydia  errors_chlamydia  \
0  45001                 125.0             162.438055        -37.438055   
1  22001                 345.0             326.619572         18.380428   
2  51001                 232.0             236.317930         -4.317930   
3  16001                2203.0            2325.000625       -122.000625   
4  19001                  27.0              26.857423          0.142577   
5  21001                  50.0              51.919460         -1.919460   
6  29001                  89.0              98.197604         -9.197604   
7  40001                 104.0             105.402461         -1.402461   
8   8001                2903.0            3057.492228       -154.492228   
9  19003                  14.0               8.982349          5.017651   

   act_values_gonorrhea  pred_values_gonorrhea  errors_gonorrhea  \
0                  58.0              68.098504        -10.098504   
1                 112.0             11